In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
import tensorflow as tf
import logging
tf.get_logger().setLevel(logging.ERROR)
import argparse
import os
from sklearn.model_selection import KFold
from monty.serialization import dumpfn, loadfn
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.losses import Huber, LogCosh
from pymatgen.core import Structure
from megnet.data.crystal import CrystalGraph, CrystalGraphWithBondTypes
from megnet.data.graph import  GaussianDistance
from megnet_models import MEGNetModel

In [3]:
def energy_within_threshold(prediction, target):
    # compute absolute error on energy per system.
    # then count the no. of systems where max energy error is < 0.02.
    e_thresh = 0.02
    error_energy = tf.math.abs(target - prediction)

    success = tf.math.count_nonzero(error_energy < e_thresh)
    total = tf.size(target)
    return success / tf.cast(total, tf.int64)

In [4]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [5]:
NAME = 'test'
SEED = 2022
CV = 5
MAX_EPOCHS = 5
BATCH_SIZE = 128
LR = 5e-4 

NFEAT_BOND = 5
CUTOFF = 4
GAUSS_CENTER = np.linspace(0, CUTOFF + 1, 100)
GAUSS_WIDTH = 0.5

In [6]:
INPUT_DIR = Path('data')
OUTPUT_DIR = Path('results') / NAME
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

# %%
cache = pd.read_pickle('data/data_cache_cutoff4.pickle')
train = pd.read_csv('data/dichalcogenides_public/targets.csv')

# %%
splitter = KFold(n_splits=CV, shuffle=True, random_state=SEED)
fold_iter = list(splitter.split(X=train, y=train['band_gap']))

In [8]:
results = []
test_ids = [ p.stem for p in (INPUT_DIR/'dichalcogenides_private/structures/').glob('*.json') ]
test_structures = [ cache.loc[cache['id'] == _id, 'structure'].values[0] for _id in test_ids ]
test_predictions = pd.DataFrame.from_dict({'id': test_ids}, orient='columns')
for fold, (train_idx, valid_idx) in enumerate(fold_iter):

    train_fold, valid_fold = train.iloc[train_idx], train.iloc[valid_idx]
    train_fold = train_fold.merge(cache, left_on='_id', right_on='id', how='left')
    valid_fold = valid_fold.merge(cache, left_on='_id', right_on='id', how='left')
    
    model = MEGNetModel(
        graph_converter=CrystalGraph(cutoff=CUTOFF),
        centers=GAUSS_CENTER,
        width=GAUSS_WIDTH,
        # loss=Huber(delta=1.0),
        loss=LogCosh(),
        npass=2,
        lr=LR,
        log_metrics=energy_within_threshold, 
        metrics_mode='min', 
        metrics='mae'
    )
    model.load_weights('data/band_gap_regression.hdf5')
    res = model.train(
        train_fold['structure'],
        train_fold['band_gap'],
        validation_structures=valid_fold['structure'],
        validation_targets=valid_fold['band_gap'],
        epochs=MAX_EPOCHS,
        batch_size=BATCH_SIZE,
        dirname=str(OUTPUT_DIR),
        filename=f'fold_{fold}',
        verbose=2,
    )

    result_fold = pd.DataFrame(loadfn(OUTPUT_DIR/f'fold_{fold}.json'))
    result_fold = pd.DataFrame(loadfn(OUTPUT_DIR/f'fold_{fold}.json'))
    result_fold['trn_loss'] = res.history.history['loss']
    result_fold['trn_energy_within_threshold'] = res.history.history['energy_within_threshold']
    result_fold[
        ['epoch', 'best_epoch', 
        'trn_loss', 'trn_energy_within_threshold', 
        'val_mae', 'val_energy_within_threshold', 'best_energy_within_threshold']].to_csv(OUTPUT_DIR/f'fold_{fold}.csv', float_format="%10.5f", index=False)
    results.append(result_fold.iloc[-1])

    model.load_weights(str(OUTPUT_DIR/f'fold_{fold}.hdf5'))
    test_predictions[f'fold_{fold}'] = model.predict_structures(test_structures)
    break

# results = pd.concat(results, axis=1)
# results.columns = np.arange(CV)
# results.to_csv(OUTPUT_DIR/'summary.csv', float_format="%10.5f")

# test_predictions['predictions'] = test_predictions[[ f'fold_{i}' for i in range(CV) ]].mean(1)
# test_predictions[['id', 'predictions']].to_csv(OUTPUT_DIR/'submission.csv', index=False)

/home/hiroshi_yoshihara/.pyenv/versions/anaconda3-2020.02/envs/idao2022/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/5


INFO:megnet_models.callbacks:val mae: 0.44289 energy_within_threshold: 0.00000
INFO:megnet_models.callbacks:best mae: inf -> 0.44289


19/19 - 48s - loss: 0.1360 - mae: 0.4692 - 48s/epoch - 3s/step


/home/hiroshi_yoshihara/.pyenv/versions/anaconda3-2020.02/envs/idao2022/lib/python3.8/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


...

INFO:megnet_models.callbacks:val mae: 0.13352 energy_within_threshold: 0.19697
INFO:megnet_models.callbacks:best mae: 0.44289 -> 0.13352


19/19 - 34s - loss: 0.0947 - mae: 0.3712 - 34s/epoch - 2s/step
Epoch 3/5


INFO:megnet_models.callbacks:val mae: 0.14762 energy_within_threshold: 0.19360


19/19 - 34s - loss: 0.0285 - mae: 0.1743 - 34s/epoch - 2s/step
Epoch 4/5


KeyboardInterrupt: 